In [38]:
import logging
import re
import sys
import openai
from dotenv import load_dotenv, find_dotenv
from llama_index.indices.knowledge_graph.base import GPTKnowledgeGraphIndex
from llama_index import ( 
    SimpleWebPageReader, 
) 
from langchain.chat_models import ChatOpenAI
from langchain.llms  import AzureOpenAI
from llama_index import StorageContext, load_index_from_storage 
from langchain.embeddings import OpenAIEmbeddings
import os

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

_ = load_dotenv(find_dotenv('.env_x'),override=True)   


if(os.environ['OPENAI_API_TYPE'] == 'azure'):
            openai.api_type = os.environ['OPENAI_API_TYPE']
            openai.api_base = os.environ['OPENAI_API_BASE']
            openai.api_version = os.environ['OPENAI_API_VERSION']
            openai.api_key = os.environ['OPENAI_API_KEY']                     
else:
            openai.api_key = os.environ['OPENAI_API_KEY']
            openai.api_base = "https://api.openai.com/v1"
            openai.api_type = os.environ['OPENAI_API_TYPE']
            openai.api_version = "" 

dataSource = ["https://support.microsoft.com/en-us/surface/surface-book-update-history-3c36b18d-1261-2cfa-4ae8-67e1a84bb175"]

urls = dataSource
   
documents = SimpleWebPageReader(html_to_text=False).load_data(urls)   
#device_name = "Intel(R) Iris(R) Plus Graphics - Extn"
current_version = "1.0.100.9000"
#device_name = "Surface UEFI - Firmware"
device_name = "Intel(R) ICLS Client - Software devices"

key_words = re.escape(device_name)
key_words = key_words.replace("\\-","[-\u2013]")

prompt = "The device driver "+device_name+" has these software versions in the list: "
for document in documents:
    titleMatch = re.search(r'<h1(.*)?>(.*)?<\/h1>',document.text, re.IGNORECASE)
    if(titleMatch):
            title = titleMatch.group(2)
    else:
            title = "Unknown"
    print("\r\n" + title + "\r\n")
    matches = re.findall(r'<p>(.*?)<\/p>[^<(.?)*]*<\/td>[^<(.?)*]*<td>[^<(.?)*]*<p>{0}<\/p>'.format(key_words), document.text, re.IGNORECASE)
    result = '\r\n'.join(matches)
    print(result)

    prompt = prompt + "\r\n" + result

print("\r\n"+prompt+"\r\n")

prompt = prompt + "\r\n\r\n" + " The software version number format is Major.Minor.Patch.MinorPatch. \
    Please compare all version numbers in the list from the first item to the last item, sort the list in descend. \
    If the largest version is 91.123.456.7890, respond in json: {\"version\":\"91.123.456.7890\"}. If the list is empty, respond this json only: {\"version\":\"0.0.0.0\"}. \
        The largest version number in the list is, I only need json string: "

messages = [{"role": "user", "content": prompt}]
response = get_completion_from_messages(messages)

print(response)






Surface Book update history

Intel – Software Component - 1.62.321.1
Intel – Software Component - 1.56.87.0

The device driver Intel(R) ICLS Client - Software devices has these software versions in the list: 
Intel – Software Component - 1.62.321.1
Intel – Software Component - 1.56.87.0

{"version":"1.62.321.1"}
